In [ ]:
import numpy as np
from imutils import paths
from sklearn.model_selection import train_test_split
import yaml
import shutil
import json
import pandas as pd
import os
import cv2

count = 0
label_dir = 'custom/json/'
txt_dir   = 'custom/txt/'
image_dir = 'custom/data/'
jpg_names = []
train_count = 0
val_count = 0
names = [ 'Left Chest', 'Right Chest', 'Full Front', 'Yoke', 'Full Back', 'Right Sleeve', 'Left Sleeve' ]

labelPaths = sorted(list(paths.list_files(label_dir)))
for i, labelPath in enumerate(labelPaths):
    content = pd.read_json(labelPath)
    filename = content.transpose()['filename'][0]
    a, b  = os.path.split(labelPath)
    c,dir_name = os.path.split(a)
    trans = content.transpose().reset_index()[['filename', 'regions']]
    x = len(trans['regions'][0])
    imagePath = image_dir +dir_name + '/'+ content.transpose()['filename'][0]
    
    if x == 0:
        if os.path.exists(imagePath) :
            os.remove(imagePath)
            os.remove(labelPath)
            print(imagePath)
        count = count + 1

    else:
        img = cv2.imread(imagePath)
        height, width = img.shape[0], img.shape[1]    
        file_name, extension = os.path.splitext(filename)

        if os.path.exists(txt_dir + dir_name) == 0:
            os.makedirs(txt_dir + dir_name)

        f = open(txt_dir + dir_name + '/' + file_name + '.txt', "w+")
        for j in range(x):
            rect_poly = trans['regions'][0][j]['shape_attributes']['name']

            dic = trans['regions'][0][j]['region_attributes']
            for index, key in enumerate(dic):
                if len(dic[key]) > 0:
                    class_number = index            
            f.write("%d " % class_number)
            if rect_poly == 'rect':
                x = trans['regions'][0][j]['shape_attributes']['x']
                y = trans['regions'][0][j]['shape_attributes']['y']
                w = trans['regions'][0][j]['shape_attributes']['width']
                h = trans['regions'][0][j]['shape_attributes']['height']

                point1_x = x / width
                point1_y = y / height
                point2_x = (x + w) / width
                point2_y = y / height
                point3_x = (x + w) / width
                point3_y = (y + h) / height
                point4_x =  x / width
                point4_y = (y + h) / height   
                f.write("%f " % point1_x)
                f.write("%f " % point1_y)
                f.write("%f " % point2_x)
                f.write("%f " % point2_y)
                f.write("%f " % point3_x)
                f.write("%f " % point3_y)
                f.write("%f " % point4_x)
                f.write("%f\r" % point4_y)
            if rect_poly == 'polygon':
                point_x = trans['regions'][0][j]['shape_attributes']['all_points_x']
                point_y = trans['regions'][0][j]['shape_attributes']['all_points_y']
                for id in range(len(point_x)):
                    f.write("%f " % float(point_x[id] / width)) 
                    f.write("%f " % float(point_y[id] / height))
                f.write("\r") 
        print(txt_dir + dir_name + '/' + file_name + '.txt')  
        txt_pos = txt_dir + dir_name + '/' + file_name + '.txt'  
        jpg_names.append(imagePath)

f.close()    
print("\n  ::::  "  + str(count) + ' images are not annotated and removed !!!   ::::  ')    
(trainX, testX) =  train_test_split(jpg_names, test_size = 0.3, random_state= 42, shuffle=True)


if os.path.exists('custom/images/train') == 0:
    os.makedirs('custom/images/train')
if os.path.exists('custom/images/val') == 0:
    os.makedirs('custom/images/val')
if os.path.exists('custom/labels/train') == 0:
    os.makedirs('custom/labels/train')
if os.path.exists('custom/labels/val') == 0:
    os.makedirs('custom/labels/val')    


f = open('custom/train.txt' , "w+")
for jpg_name in trainX:
    jpg_src_path = jpg_name
    head_tail = os.path.split(jpg_src_path)
    base, dir_name = os.path.split(head_tail[0])
    file_name, extension = os.path.splitext(head_tail[1])
    jpg_dst_path = 'custom/images/train/' 
    shutil.copy(jpg_src_path, jpg_dst_path)

    txt_src_path = 'custom/txt/' + dir_name + '/' + file_name + '.txt'
    txt_dst_path = 'custom/labels/train/' 
    shutil.copy(txt_src_path, txt_dst_path)
    train_count += 1

    save_dir = '../custom/images/train/' + file_name + '.jpg'
    f.write("%s " % save_dir)
    f.write("\r")
f.close()    

f = open('custom/val.txt' , "w+")
for jpg_name in testX:
    jpg_src_path = jpg_name
    head_tail = os.path.split(jpg_src_path)
    base, dir_name = os.path.split(head_tail[0])
    file_name, extension = os.path.splitext(head_tail[1])
    jpg_dst_path = 'custom/images/val/' 
    shutil.copy(jpg_src_path, jpg_dst_path)

    txt_src_path = 'custom/txt/' + dir_name + '/' + file_name + '.txt'
    txt_dst_path = 'custom/labels/val/' 
    shutil.copy(txt_src_path, txt_dst_path)   
    val_count += 1

    save_dir = '../custom/images/val/' + file_name + '.jpg'
    f.write("%s " % save_dir)
    f.write("\r")
f.close()

print('\n' + str(train_count)+ 'images are training data.  ')
print(str(val_count)+ 'images are validation data.  ')    

d = {
        'train': '../custom/train.txt  # train images (relative to path) 128 images',
        'val': '../custom/val.txt  # val images (relative to path) 128 images',

        'nc': len(names),
        'names': names }  # dictionary

with open('polygon-yolov5/data/custom.yaml', 'w') as f:
    yaml.dump(d, f, sort_keys=False)